In [27]:
# this code implements a deep neural network for feature learning
# this code comes from Vishwa's class on constructing a data loader and creating a model

# Some common system imports
import os
import sys
import importlib
import time
import csv

# Numeric computing
import numpy as np

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from scipy.io import wavfile

# pytorch
import torch
import torch.utils.data as tdata
from torchaudio import transforms
import torch.utils.data as data_utils

# Plotting (if we want it)
import matplotlib.pyplot as plt

# importing our own modules
import audio_datasets as ads

# for one-hot vectore
import torch.nn.functional as func

#PCA packages
from sklearn.datasets import load_digits
from sklearn.decomposition import KernelPCA

#chroma import
import librosa

In [2]:
# loading the raw data for feature learning
training_path = os.path.join(os.getcwd(), "..", "training_data/data") #need to change this back before pushing ***
files = os.listdir(training_path)
wav_files = []
file_type = "wav"
LABELS = {"neutral": 0, "calm": 1, "happy": 2, "sad": 3, "angry": 4, "fearful": 5, "disgust": 6, "surprised": 7}

for file in files:
    curr_path = os.path.join(training_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        for label in LABELS.keys():
            if label in file:
                wav_files.append((file, LABELS[label]))

data_array = []
label_array = []
for data in wav_files: #in the my_datasets code
    data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])
    label_array.append(data[1])

max_len = max([len(data) for data in data_array])
data_array = [np.resize(data, 48000*5) for data in data_array]
print(len(data_array[0]))

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_2588/3951241112.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])


240000


In [14]:
# convolution layer
# structure comes from IBM website
# will use mel_coef as feautre vectors

#mfcc_features = torch.tensor(mfcc_features)

CONV_KERNEL = 3
conv_layer1 = torch.nn.Conv1d(12, 12, CONV_KERNEL)
conv_layer2 = torch.nn.Conv1d(12, 12, CONV_KERNEL)
#conv_layer3 = torch.nn.Conv1d(50, 50, CONV_KERNEL)
#conv_layer2 = torch.nn.
#conv_layer3 = 
#conv_out = conv_layer(mfcc_features)

#print(conv_out.shape)



In [15]:
# pooling layer
KERNEL_SIZE = 3
pooling_layer = torch.nn.MaxPool1d(KERNEL_SIZE)

#pooled_out = pooling_layer(conv_out)

#pooled_out = pooled_out.flatten()
#pooled_out.shape

In [16]:
# fully connected layer
#implementing kernel SVM
class MLP(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(in_features=17100, out_features=500)
    self.fc2 = torch.nn.Linear(in_features=500, out_features=100)
    self.fc3 = torch.nn.Linear(in_features=100, out_features=8)
    #self.fc4 = torch.nn.Linear(in_features=10, out_features=8)

  def forward(self, x):
    x1 = torch.relu(self.fc1(x))
    x2 = torch.relu(self.fc2(x1))
    x3 = self.fc3(x2)
    #x4 = self.fc4(x3) #usually left linear
    return torch.sigmoid(x3) #for classification

In [33]:
# split into training and testing
#transformer = KernelPCA(n_components=40, kernel='cosine')
chroma_data_array = []
for data in data_array:
    chroma_data = librosa.feature.chroma_stft(data.astype(np.float32), sr=48000)
    chroma_data_array.append(np.array(chroma_data))
train_data, test_data, train_labels, test_labels = train_test_split(chroma_data_array, label_array, train_size=0.75, test_size=0.25)
#train_data = mfcc_data_array
#train_labels = label_array
#print(data.shape, train_data.shape, test_data.shape)

#train_ten, test_ten = torch.tensor(train_data), torch.tensor(test_data)
train_ten = torch.from_numpy(np.array(train_data)).double()
test_ten = torch.from_numpy(np.array(test_data)).double()
train_y_ten, test_y_ten = torch.tensor(train_labels), torch.tensor(test_labels)
print(train_ten.shape)
train_dataset = tdata.TensorDataset(train_ten, train_y_ten)

train_loader = tdata.DataLoader(train_dataset, batch_size=50) #this is the data loader (just doing 1 point at a time)

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_2588/1360374942.py:5: FutureWarning: Pass y=[ 2.  2.  2. ... 30. 27. 22.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_data = librosa.feature.chroma_stft(data.astype(np.float64), sr=48000)
/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_2588/1360374942.py:5: FutureWarning: Pass y=[  0.   0.   0. ...  46.  69. -23.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_data = librosa.feature.chroma_stft(data.astype(np.float64), sr=48000)
/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_2588/1360374942.py:5: FutureWarning: Pass y=[  -2.   -2.   -3. ... -370.   60.  387.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_data = librosa.feature.chroma_stft(data.astype(np.float64), sr=48000)
/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipyker

torch.Size([843, 12, 469])


/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_2588/1360374942.py:5: FutureWarning: Pass y=[   0.    0.   -1. ... -423. -130.  337.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  chroma_data = librosa.feature.chroma_stft(data.astype(np.float64), sr=48000)


In [34]:
print(train_ten.shape)
print(test_ten.shape)
print(train_y_ten.shape)
print(test_y_ten.shape)

torch.Size([843, 12, 469])
torch.Size([282, 12, 469])
torch.Size([843])
torch.Size([282])


In [35]:
n_epochs = 10 #essentially number of iterations

#instantiating a model
model = MLP()

#loss function
criterion = torch.nn.CrossEntropyLoss() #binary cross-entropy loss, clamps log values to protect against extremes

#optimizer mechanism
optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())

loss_array = np.zeros(n_epochs)

#something about test accuracy here - see final code
for epoch_idx in range(n_epochs):
  print(epoch_idx)
  for batch in train_loader:
    data_batch, labels_batch = batch
    
    # convolutional layer
    conv_out = conv_layer1(data_batch)
    conv_out = conv_layer2(conv_out)
    #conv_out = conv_layer3(conv_out)
    relu_conv = torch.relu(conv_out)
    #print("1", conv_out.shape)
    # pooling layer
  
    pooled_out = pooling_layer(relu_conv)
    #print("2", pooled_out.shape)
    fc_layer_in = pooled_out.reshape((pooled_out.shape[0], -1))
    # move to GPU if available - data_batch = data_batch.cuda() etc.
    #predict
    #print("3",fc_layer_in.shape)
    labels_pred = model(fc_layer_in.to(torch.float32))
   
    #print(labels_pred.shape)
    #compute loss
    loss = criterion(labels_pred, labels_batch) #note: cross entropy is not symmetric

    loss_array[epoch_idx] += loss.item()/data_batch.shape[0] #mean of loss
    #plotting the precision
    #with torch.no_grad():
      #test_pred = model(test_ten).flatten()
      #test_loss =
 
    #backprop
    optimizer.zero_grad()
    loss.backward() #backprop from pytorch
    optimizer.step() #all params optimized
  #if epoch_idx%50 == 0:
  #  fig.data[0].y = loss_array[:epoch_idx]
  #  fig.update_yaxes(type='log')

0


RuntimeError: expected scalar type Double but found Float

In [81]:
# loading the raw data for feature learning
test_path = os.path.join(os.getcwd(), "..", "test_data")
test_files = os.listdir(test_path)
test_wav_files = []

for file in test_files:
    curr_path = os.path.join(test_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        test_wav_files.append(file)

test_data_array = []
for data in test_wav_files: #in the my_datasets code
    test_data_array.append(wavfile.read(os.path.join(test_path, data))[1])

test_data_array = [np.resize(data, 48000*5) for data in test_data_array]

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_87777/2716096835.py:13: WavFileWarning: Chunk (non-data) not understood, skipping it.
  test_data_array.append(wavfile.read(os.path.join(test_path, data))[1])


In [84]:
# split into training and testing
mfcc_test_data_array = [np.array(mfcc(torch.tensor(data).float())) for data in test_data_array] #transform into mfcc before model
test_data = mfcc_test_data_array
#print(data.shape, train_data.shape, test_data.shape)

test_ten = torch.tensor(test_data)
test_dataset = tdata.TensorDataset(test_ten)

test_loader = tdata.DataLoader(test_dataset, batch_size=50)

In [71]:
test_pred = []
test_dataset = tdata.TensorDataset(test_ten, test_y_ten)
test_loader = tdata.DataLoader(test_dataset, batch_size=50) 
with torch.no_grad():
  for test in test_loader:
    # convolutional layer
    conv_test = conv_layer1(test[0])
    conv_test = conv_layer2(conv_test)
    #conv_test = conv_layer3(conv_test)
    relu_conv_test = torch.relu(conv_test)

    # pooling layer
    pooled_test = pooling_layer(relu_conv_test)
    fc_layer_test = pooled_test.reshape((pooled_test.shape[0], -1))
    
    label_histogram = model(fc_layer_test.to(torch.float32))
    for histogram in label_histogram:
      test_pred.append(np.argmax(histogram).item())
  
test_pred = torch.tensor(test_pred)
print(classification_report(test_y_ten, test_pred))

'''
LABEL_CONVERTER = dict([(value, key) for key, value in LABELS.items()])

#counting the mislabeled points
with open("cnn_results.csv", "w+") as result_file:
    writer = csv.writer(result_file)
    writer.writerow(["filename", "label"])

    for idx in range(len(test_pred)):
        writer.writerow([os.path.splitext(test_wav_files[idx])[0], LABEL_CONVERTER[test_pred[idx]]])

#print(classification_report(test_y_ten, test_pred))
#print("Number of mislabeled points out of a total %d points : %d", (test_ten.shape[0], (test_y_ten != y_pred).sum()))
'''

              precision    recall  f1-score   support

           0       0.75      0.17      0.27        18
           1       0.44      0.59      0.51        32
           2       0.42      0.47      0.44        38
           3       0.23      0.30      0.26        43
           4       0.52      0.50      0.51        34
           5       0.55      0.33      0.41        36
           6       0.51      0.37      0.43        49
           7       0.36      0.50      0.42        32

    accuracy                           0.41       282
   macro avg       0.47      0.40      0.41       282
weighted avg       0.45      0.41      0.41       282



'\nLABEL_CONVERTER = dict([(value, key) for key, value in LABELS.items()])\n\n#counting the mislabeled points\nwith open("cnn_results.csv", "w+") as result_file:\n    writer = csv.writer(result_file)\n    writer.writerow(["filename", "label"])\n\n    for idx in range(len(test_pred)):\n        writer.writerow([os.path.splitext(test_wav_files[idx])[0], LABEL_CONVERTER[test_pred[idx]]])\n\n#print(classification_report(test_y_ten, test_pred))\n#print("Number of mislabeled points out of a total %d points : %d", (test_ten.shape[0], (test_y_ten != y_pred).sum()))\n'